In [5]:
def process_file(file):
    col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]
    
    for i in range(1, 33):
        col_names.append(f"Sp{i}")
    
    df = pd.read_csv(file, names=col_names, sep='\s+', header=None)
    df = df.set_index('Index')
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
    df = df[df["MonStatus"] == 1]

    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    df['DateM'] = df['DateM'].str[:3].map(month_map)
    df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
    df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')

    df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
                                     month=df['Date'].dt.month,
                                     day=df['Date'].dt.day,
                                     hour=df['Time'].dt.hour,
                                     minute=df['Time'].dt.minute,
                                     second=df['Time'].dt.second))

    df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)

    day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}

    df.insert(0, 'Day', df['Time'].dt.day.map(day_map))
    
    return df

In [11]:
def mins(time):
    if isinstance(time, pd.Series):
        return time.dt.hour * 60 + time.dt.minute
    else:
        return time.hour * 60 + time.minute
        
def get_masking_bounds(df, window_length_mins = 120):
    times = df['Time']
    
    lights = df['Light']
    
    light_off_times = list(df['Time'][(lights == 0) & (lights.shift(1) == 1)])
    light_on_times = list(df['Time'][(lights == 1) & (lights.shift(1) == 0)])

    first_off = light_off_times[0]
    first_on = light_on_times[0]

    off_bound = max(light_off_times, key=lambda t: abs(mins(t) - mins(first_off)))
    on_bound = max(light_on_times, key=lambda t: abs(mins(t) - mins(first_on)))
    
    return sorted((off_bound, on_bound))
    
def zt_graph(df, spiders, title, masked=False, rolling_window = 1):
    df = df.reset_index(drop=True)
    lights = df['Light']
    
    light_on_times = df.index[(lights == 1) & (lights.shift(1) == 0)]
    first_on_time = df['Time'][light_on_times[0]]

    zt_sum = np.zeros(1440)
    
    if not isinstance(spiders, list):
        spiders = [spiders]

    mask_bounds = get_masking_bounds(df)
    
    for on_time in light_on_times:
        if df['Time'][on_time] in mask_bounds:
            continue
        for spider in spiders:
            section = df[spider][on_time : on_time + 1440]
            zt_sum[:len(section)] += np.array(section)
    
    fig, ax = plt.subplots()

    zts = df[light_on_times[0] : light_on_times[0] + 1440]
    x_time = (zts.index - light_on_times[0]) / 60

    zt_avg = zt_sum / len(light_on_times) / len(spiders)

    zt_avg_smooth = pd.Series(zt_avg).rolling(rolling_window).mean().fillna(0)

    ax.plot(x_time, zt_avg_smooth)
    ax.fill_between(x_time, 0, max(zt_avg_smooth), where=(zts['Light'] == 1), color='yellow', alpha=.5)
    if (masked):
        ax.fill_betweenx([0, max(zt_avg_smooth)], ((mins(mask_bounds[0]) - mins(first_on_time)) / 60) % 24, ((mins(mask_bounds[1]) - mins(first_on_time)) / 60) % 24, color='red', alpha=.25)

    fig.suptitle(title)
    fig.supxlabel('ZT Time (Hours)')
    fig.supylabel('Activity averaged over all days')
    
    return fig, ax

In [12]:
def rasterplot_binary(dataframe, spider_name, title, rolling_window=1):
    sp_df = dataframe[['Day', 'Time', 'Light', spider_name]]
    
    days = sp_df['Day'].unique()
    
    fig, axs = plt.subplots(len(days), 1)
    
    fig.suptitle(title)
    fig.supxlabel('Time (hrs)')
    fig.supylabel('Day')
    
    # Separate activity by day
    for day, ax in zip(days, axs):
        sp_day = sp_df[sp_df['Day'] == day]
        time = sp_day['Time'].dt.hour + sp_day['Time'].dt.minute / 60
        light = sp_day['Light']
        activity = sp_day[spider_name]
    
        ax.tick_params(left=False, bottom=False)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        ax.set_xlim(-0.5, 23.5)
        ax.set_ylim(0, 1)
    
        ax.set_ylabel(day, rotation='horizontal')

        ax.fill_between(time, 0, 1, where=light, alpha=.5, color='yellow')
        ax.fill_between(time, 0, 1, where=(activity.rolling(rolling_window).mean() > 0), alpha=1)
    
    axs[-1].set_xticks(np.arange(0, 25, 2))
    axs[-1].set_xticklabels(np.arange(0, 25, 2), rotation = 'horizontal')

    return fig, axs